<a href="https://colab.research.google.com/github/Damilola-ops/Text-de-shuffling/blob/main/Text_de_shuffling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets
!pip install pandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.8 MB/s 
     |████████████████████████████████| 362 kB 65.5 MB/s 
     |████████████████████████████████| 596 kB 63.4 MB/s 
     |████████████████████████████████| 101 kB 15.1 MB/s 
     |████████████████████████████████| 6.6 MB 63.6 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 1.1 MB 54.3 MB/s 
     |████████████████████████████████| 140 kB 15.5 MB/s 
     |████████████████████████████████| 127 kB 56.5 MB/s 
     |████████████████████████████████| 144 kB 55.9 MB/s 
     |████████████████████████████████| 271 kB 61.6 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existin

In [43]:
from transformers import EncoderDecoderModel, EncoderDecoderConfig, BertTokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertConfig
from transformers import BartConfig,BartTokenizerFast,BartForCausalLM,BartModel
import torch
import pandas as pd 
import numpy as np
import sklearn.preprocessing 
from torch.nn import functional as F 
import datasets
from datasets import load_dataset


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv')

val_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv')

In [23]:
train_df.columns

Index(['text', 'label', 'id'], dtype='object')

In [31]:
#using huggingface's dataset function to load multiple datasets at once 
dataset = load_dataset( 'csv',data_files = {'train':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv'],
                                         'val':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv'],
                                         'test':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv']})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f3bbdea8f3573fa9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
train_df['label'].loc[5]

'Surprisingly, the transition is fastest with an intermediate flexibility of the hinge.'

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 4001
    })
    val: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 4001
    })
})

In [11]:
bart_tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

In [13]:
text = 'offisong is a fool '
tokens = bart_tokenizer(text)
tokens

{'input_ids': [0, 1529, 354, 1657, 16, 10, 17275, 1437, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
train_df.columns


Index(['label', 'id'], dtype='object')

In [36]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [37]:
bert_tokens = tokenizer.tokenize(sample)
bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)
print('tokens: ', bert_tokens)
print('ids: ',bert_ids)


tokens:  ['surprisingly', ',', 'the', 'transition', 'is', 'fastest', 'with', 'an', 'intermediate', 'flexibility', 'of', 'the', 'hi', '##nge', '.']
ids:  [10889, 1010, 1996, 6653, 2003, 7915, 2007, 2019, 7783, 16991, 1997, 1996, 7632, 15465, 1012]


In [35]:
sample = train_df['label'].loc[5]
tokens = bart_tokenizer.tokenize(sample)
token_id = bart_tokenizer.convert_tokens_to_ids(tokens)
print('sample text: ', sample)
print('tokens : ',tokens)
print('token_id : ',token_id)

sample text:  Surprisingly, the transition is fastest with an intermediate flexibility of the hinge.
tokens :  ['Sur', 'prisingly', ',', 'Ġthe', 'Ġtransition', 'Ġis', 'Ġfastest', 'Ġwith', 'Ġan', 'Ġintermediate', 'Ġflexibility', 'Ġof', 'Ġthe', 'Ġhinge', '.']
token_id :  [27526, 47607, 6, 5, 3868, 16, 6273, 19, 41, 21398, 8243, 9, 5, 33503, 4]


In [38]:
original_text = bart_tokenizer.decode(token_id)
original_text

'Surprisingly, the transition is fastest with an intermediate flexibility of the hinge.'

In [39]:
#a function to tokenize all the rows in our dataset
max_text_length = 40 
max_label_length = 40 
def preprocess_function(sample):
    # Getting text and label
    text = sample["text"]
    label = sample["label"]
    # Tokenizing the text and label
    inputs = bart_tokenizer(text, padding="max_length", truncation=True, max_length=max_text_length)
    outputs = bart_tokenizer(label, padding="max_length", truncation=True, max_length=max_label_length)
    sample["input_ids"] = inputs.input_ids
    sample["attention_mask"] = inputs.attention_mask
    sample["decoder_input_ids"] = outputs.input_ids
    sample["decoder_attention_mask"] = outputs.attention_mask
    sample["labels"] = outputs.input_ids
    # The labels are used to calcuate the loss while training, and because we added padding to make all tokens to be of same size,
    # we also need to convert the padding number ( 0 ) to ( -100 ), so that we can tell huggingface that these number can be ignorned while calcuating loss. 
    # Why specifically -100 ? It's simply an arbitrary number, again so that huggingface can ignore this number while calcuating loss
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]

    return sample

In [40]:
batch = 16
tokenized_dataset=dataset.map(preprocess_function,batch_size=batch,batched=True)

  0%|          | 0/2501 [00:00<?, ?ba/s]

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/251 [00:00<?, ?ba/s]

In [42]:
tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [45]:
model = BartForCausalLM.from_pretrained('facebook/bart-base')

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartForCausalLM: ['encoder.layers.4.final_layer_norm.weight', 'encoder.layers.0.self_attn.out_proj.weight', 'encoder.layers.2.self_attn.out_proj.weight', 'encoder.layers.1.fc1.bias', 'encoder.layers.5.self_attn.k_proj.bias', 'encoder.layers.5.self_attn.out_proj.weight', 'encoder.layers.2.fc2.weight', 'encoder.layers.3.self_attn.q_proj.bias', 'encoder.layers.4.self_attn_layer_norm.weight', 'encoder.layers.2.self_attn.k_proj.bias', 'encoder.layers.3.final_layer_norm.bias', 'encoder.layers.3.self_attn.k_proj.weight', 'encoder.layers.5.self_attn_layer_norm.weight', 'encoder.layers.2.self_attn.q_proj.bias', 'encoder.layers.2.self_attn.out_proj.bias', 'encoder.layers.5.final_layer_norm.bias', 'encoder.layers.0.self_attn.v_proj.weight', 'encoder.layers.5.fc1.weight', 'encoder.layers.4.fc2.bias', 'encoder.layers.0.self_attn.k_proj.bias', 'encoder.layers.3.fc2.weight', 'encoder.layers.5.fc1.bias', 'encode

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
model#model architecture

In [80]:
model.config.decoder_start_token_id = bart_tokenizer.cls_token_id
model.config.eos_token_id = bart_tokenizer.sep_token_id
model.config.pad_token_id = bart_tokenizer.pad_token_id
model.config.vocab_size = model.config.vocab_size


In [81]:
N_epochs = 1
args = Seq2SeqTrainingArguments(
    "Scambled Text",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    num_train_epochs=N_epochs,
    fp16=True, # This will help in increasing the speed of training
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [82]:
trainer = Seq2SeqTrainer(model = model,
                         args = args,
                         train_dataset = tokenized_dataset['train'],
                         eval_dataset = tokenized_dataset['val']
                        )

                                            
                         
                         

Using cuda_amp half precision backend


In [83]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForCausalLM.forward` and have been ignored: decoder_attention_mask, text, decoder_input_ids, id. If decoder_attention_mask, text, decoder_input_ids, id are not expected by `BartForCausalLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40001
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2501


Epoch,Training Loss,Validation Loss
1,2.309600,3.027571


Saving model checkpoint to Scambled Text/checkpoint-500
Configuration saved in Scambled Text/checkpoint-500/config.json
Model weights saved in Scambled Text/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-1000
Configuration saved in Scambled Text/checkpoint-1000/config.json
Model weights saved in Scambled Text/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-1500
Configuration saved in Scambled Text/checkpoint-1500/config.json
Model weights saved in Scambled Text/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-2000
Configuration saved in Scambled Text/checkpoint-2000/config.json
Model weights saved in Scambled Text/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-2500
Configuration saved in Scambled Text/checkpoint-2500/config.json
Model weights saved in Scambled Text/checkpoint-2500/pytorch_model.bin
The following columns in the evaluatio

TrainOutput(global_step=2501, training_loss=2.2675581270101213, metrics={'train_runtime': 236.5786, 'train_samples_per_second': 169.081, 'train_steps_per_second': 10.572, 'total_flos': 544450654126080.0, 'train_loss': 2.2675581270101213, 'epoch': 1.0})

In [64]:
def generate_predictions(batch):

    # Tokenizing the test
    inputs = bart_tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_text_length, return_tensors="pt")
    
    # Sending the tensors to GPU
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # Generating the predicted tokens ids
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # Converting the token ids to sentence
    output_str = bart_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["predictions"] = output_str

    return batch
    
    
    

In [ ]:
input_ids = bert_tokens.input_ids
attention_mask = bert_tokens.attention_mask
outputs = model

In [79]:
result = dataset['test'].map(generate_predictions,batch_size = batch,batched=True)

  0%|          | 0/251 [00:00<?, ?ba/s]

Input length of input_ids is 40, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


RuntimeError: ignored

In [91]:
test_text = test_df['text'].loc[5]
input = bart_tokenizer(test_text,return_tensors= 'pt')
input_id = bart_tokenizer.convert_tokens_to_ids(input)
attention_mask = input.attention_mask
output = model.generate((input['input_ids']).to('cuda'))
output = tokenizer.decode(output)


TypeError: ignored

In [72]:
test_text

'Most vascular Hessian-based enhancing techniques often well-known rely filters. on'

In [ ]:
test_df.head(5)

,id,text,label
0,0,safely objects move. that system images detect...,"system Using approach, move. safely skip of th..."
1,1,We detectors popular influence of confidences ...,confidences popular different in detectors We ...
2,2,compact coding present supervised We a approac...,"We a supervised approach, compact present codi..."
3,3,study high-throughput vital of quantitative be...,is for and individuals study of of collective ...
4,4,on data sets. We evaluate method many challeng...,sets. challenging the data We method on many e...
